In [28]:
!pip install -q -U langchain_community langchain_core langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.0 MB/s eta 0:00:00


In [45]:
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_core.tools import tool
import requests
from langchain_core.messages import HumanMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [ ]:
@tool
def get_conversion_factor(base_currency : str, target_currency : str) -> float:
    """This function fetches the currency conversion factor between
    given base currency & target currency"""

    url = f"https://v6.exchangerate-api.com/v6/02cd6e6a6bd5ef0cd3b8b39b/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()['conversion_rate']


@tool
def convert(base_currency_value : int, conversion_rate : Annotated[float, InjectedToolArg]) -> float:
    """Given a currency conversion rate and base currency, this function calculates the target currency value"""
    return base_currency_value * conversion_rate



In [47]:
rate = get_conversion_factor.invoke({"base_currency" : "USD", "target_currency" : "BDT"})

In [48]:
convert.invoke({"base_currency_value" : 50, "conversion_rate" : rate})

6084.81

In [ ]:
llm = ChatOpenAI()

In [21]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [43]:
message = [HumanMessage("what is the conversion rate of USD to BDT, and can you convert 50 USD to BDT")]

In [ ]:
llm_with_tools.invoke(message)

In [ ]:
llm_with_tools.invoke(message).tool_calls

In [ ]:
ai_message = llm_with_tools.invoke(message)

In [ ]:
message.append(ai_message)

In [ ]:
import json

for tool_call in ai_message.tool_calls:
    if tool_call['name'] == "get_conversion_factor":
        tool_message1 = get_conversion_factor.invoke(tool_call)

        conve_rate = json.load(tool_message1.content)['conversion_rate']
        message.append(tool_message1)

    if tool_call['name'] == "convert":
        tool_call['args']['conversion_rate']= conve_rate

        tool_message2 = convert.invoke(tool_call)
        message.append(tool_message2)


In [ ]:
message

In [ ]:
llm_with_tools.invoke(message)